<a href="https://colab.research.google.com/github/riccardomarin/EG22_Tutorial_Spectral_Geometry/blob/main/forward/03_Shape_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One of the principal application of the spectral shape analysis is shape matching! Here we provide a complete tour from the basics to the state of the art refinement involved

In [ ]:
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/data/tr_reg_090.off
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/data/tr_reg_043.off
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/utils/utils_mesh.py
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/utils/utils_spectral.py

!pip install plyfile

import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import scipy.io as sio
import scipy.sparse.linalg
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix

import numpy as np
import pandas as pd
import torch  
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
from torch import nn
from torch.autograd import Variable

from utils_spectral import LB_FEM_sparse, EigendecompositionSparse, LB_cotan, Eigendecomposition
from utils_mesh import load_off
from utils_mesh import plot_colormap, plot_RGBmap

First of all, we load two shapes

In [ ]:
with open("tr_reg_090.off") as f:
  v_src, f_src = load_off(f)
  f_src = np.asarray(f_src).astype('long')

with open("tr_reg_043.off") as f:
  v_tar, f_tar = load_off(f)
  f_tar = np.asarray(f_tar).astype('long')

p = plot_colormap([v_src,v_tar], [f_src, f_tar],[np.ones(v_src.shape[0])]*2)
p.show()

These shapes are naturally in correspondence, since they share the same connectivity (trinagulation). Hence, we can define a function on one shape, and use this correspondence to transfer the function on the other.

In [ ]:
funz_ = (v_tar - np.min(v_tar,0))/np.tile((np.max(v_tar,0)-np.min(v_tar,0)),(np.size(v_tar,0),1));
colors = np.cos(funz_);
funz_tar = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_tar,funz_tar])
p.show()

# Matching by descriptors

# WKS

We will use the Wave Kernel Signature (WKS) descriptor to do the matching. Recall the formula:

$K_E(x,x) = \sum\limits_{l=1}^{\infty}e^{- \frac{(log(E) - log(\lambda_l))^2}{2\sigma^2}} \phi_l(x)^2 $

Where:
- $sigma = 7 \delta$
- $delta =  (e_{max} - e{min})/ M$
- $e_{max} = log(E_N) - 2\sigma$
- $e_{min} = log(E_1) + 2\sigma$
- $E_N$ is the max eigenvalue in absolute value
- $E_1$ is the min non-zero eigenvalue in absolute value
- $M$ is the number of WKS scales

The steps are:
- Reading the meshes, computing the LBO eigenvectors
- Defining the WKS computation
- Visualizing the WKS scales on meshes
- Performing the matching using WKS (Nearest-Neighbor in the descriptor space)
- Visualizing the matching (and computing the error)

Are the descriptors coherent among the shapes, for different descriptor scales? Is the matching good? We can change the number of descriptors: does it impact the matching?

In [ ]:
# compute the WKS descriptor
def WKS(vertices, faces, evals, evecs, wks_size, variance):
    # Number of vertices
    n = vertices.shape[0]
    WKS = np.zeros((n,wks_size))

    # Just for numerical stability
    evals[evals<1e-6] = 1e-6

    # log(E)
    log_E = np.log(evals).T

    # Defining the energies step
    e = np.linspace(log_E[1], np.max(log_E)/1.02, wks_size)

    # Computing the sigma
    sigma = (e[1]-e[0]) * variance
    C = np.zeros((wks_size,1))

    for i in np.arange(0,wks_size):
        # Computing WKS
        WKS[:,i] = np.sum(
            (evecs)**2 * np.tile( np.exp((-(e[i] - log_E)**2) / (2*sigma**2)),(n,1)), axis=1)
        
        # Normalization
        C[i] = np.sum(np.exp((-(e[i]-log_E)**2)/(2*sigma**2)))
        
    WKS = np.divide(WKS,np.tile(C,(1,n)).T)
    return WKS

In [ ]:
dtype = 'float32'
k = 100

# === COMPUING LBO EIGENFUNCTIONS ===
v_src_t = torch.from_numpy(v_src.astype(dtype)).cuda()*1
f_src_t = torch.from_numpy(np.asarray(f_src).astype('long')).cuda()

L_sym_sp, A_sp_src, Ainv_sp = LB_FEM_sparse(v_src_t, f_src_t.long())
evecs_src, evals_src = EigendecompositionSparse(L_sym_sp.values(),L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs_src = evecs_src * Ainv_sp[:,None]

v_tar_t = torch.from_numpy(v_tar.astype(dtype)).cuda()*1
f_tar_t = torch.from_numpy(np.asarray(f_tar).astype('long')).cuda()

L_sym_sp, A_sp_tar, Ainv_sp = LB_FEM_sparse(v_tar_t, f_tar_t.long())
evecs_tar, evals_tar = EigendecompositionSparse(L_sym_sp.values(),L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs_tar = evecs_tar * Ainv_sp[:,None]

# === SAVING IN NUMPY ===
evecs_tar = evecs_tar.detach().cpu().numpy()
evals_tar = evals_tar.detach().cpu().numpy()

evecs_src = evecs_src.detach().cpu().numpy()
evals_src = evals_src.detach().cpu().numpy()

In [ ]:
import scipy as sp

# Computing the descriptors for the two shapes
d_src = WKS(v_src, f_src, evals_src, evecs_src, 100, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, 100, 7)

# Nearest Neighbor to obtain the p2p map
treesearch = sp.spatial.cKDTree(d_src)
p2p = treesearch.query(d_tar, k=1)[1]

# To see the quality of the matching we plot a function on one shape and we transfer it to the other
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p,:]])
p.show()

# Computing (euclidean) error evaluation
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

# Functional Maps

Instead to directly match the point signatures, we can use them to align the eigenfunctions of the two shapes. 

Here we deploy a basic optimization for Functional Maps framework, to recover the functional map $C$. We optimize $C$ for the following loss:

$ Loss(C) = w_1 \|CA - B \|_2 + w_2 \|C\Lambda_{src} - \Lambda_{tar}C\|$

The first term is the descriptor preservation term (i.e., $C$ should map the  source shape descriptors' coefficients $A$ to the correct coefficient $B$ on the target shape), and the second term is the commutativity with the Laplacian (that promotes isometric maps).

In [ ]:
# Recovering useful variables
evecs_tar = evecs_tar
evals_tar = evals_tar
evecs_src = evecs_src
evals_src = evals_src
A_src = np.diag(A_sp_src.cpu().detach().numpy())
A_tar = np.diag(A_sp_tar.cpu().detach().numpy())

# Computing descriptors
n_land =  6

n_wks  =  20
n_evals = 20

# Landmarks, as step functions randomly sampled
step = np.int(np.ceil(v_src.shape[0] / n_land))
a = np.arange(0,v_src.shape[0],step)
landmarks = np.zeros((v_src.shape[0], a.size))
landmarks[a,np.arange(a.size)] = 1

# WKS
d_src = WKS(v_src, f_src, evals_src, evecs_src, n_wks, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, n_wks, 7)

# Optimization Process
desc_src = np.hstack((landmarks,d_src))
desc_tar = np.hstack((landmarks,d_tar))

In [ ]:
# Descriptor normalization
no = np.sqrt(np.diag(np.matmul(A_src.__matmul__(desc_src).T, desc_src)))
no_s = np.tile(no.T,(v_src.shape[0],1))
no_t = np.tile(no.T,(v_tar.shape[0],1))
fct_src = np.divide(desc_src,no_s)
fct_tar = np.divide(desc_tar,no_t)

# Coefficents of the obtained descriptors
Fct_src = np.matmul(A_src.T.__matmul__(evecs_src[:, 0:n_evals]).T, fct_src)
Fct_tar = np.matmul(A_tar.T.__matmul__(evecs_tar[:, 0:n_evals]).T, fct_tar)

# The relation between the two constant functions can be computed in a closed form
constFct = np.zeros((n_evals,1))
constFct[0, 0] = np.sign(evecs_src[0, 0] * evecs_tar[0, 0]) * np.sqrt(np.sum(A_tar)/np.sum(A_src))

In [ ]:
# Energy weights
w1 = 1e-1 # Descriptors preservation
w2 = 1e-8 # Commutativity with Laplacian

# Define tensorflow objects
fs = torch.Tensor(Fct_src)
ft = torch.Tensor(Fct_tar)
evals = torch.diag(torch.Tensor(np.reshape(np.float32(evals_src[0:n_evals]), (n_evals,))))
evalt = torch.diag(torch.Tensor(np.reshape(np.float32(evals_tar[0:n_evals]), (n_evals,))))

We are ready to run the optimization!

In [ ]:
import progressbar
C_ini = np.zeros((n_evals,n_evals))
C_ini[0,0]=constFct[0,0]
C = Variable(torch.Tensor(C_ini), requires_grad=True)

optimizer = torch.optim.Adam([C], lr=5e-2)

for it in progressbar.progressbar(range(1500)):   
    optimizer.zero_grad()

    # Loss computation
    loss1 = w1 * torch.sum(((torch.matmul(C, fs) - ft) ** 2)) / 2 # Descriptor preservation
    loss2 = w2 * torch.sum((torch.matmul(C, evals) - torch.matmul(evalt,C))**2) # Commute with Laplacian
    loss = torch.sum(loss1  + loss2)

    # Gradient descent
    loss.backward()
    optimizer.step()

Now we can visualize the obtained $C$ and the correspondence. To compute the correspondence we can use the obtained $C$ to transfer the delta functions and then perform Nearest-Neighbor in the space of the aligned bases.

In [ ]:
# Showing C matrix
C_np = C.detach().numpy()
plt.imshow(C_np)
plt.colorbar()
plt.show()

# Point-to-point correspondence computation
treesearch = sp.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals], C_np.T))
p2p = treesearch.query(evecs_tar[:,0:n_evals], k=1)[1]

# Correspondence visualization
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p,:]])
p.show()

# Computing (euclidean) error evaluation
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

Notice that FMAP matching is much smoother than the one obtained by WKS, and the numerical error significantly decreases!

## ICP Refinement
The correspondence can be post-processed by ICP registration in the eigenfunction space!

In [ ]:
print('ICP refine...')
C_ICP = C_np

# ICP Refining
for k in np.arange(0,5):
    treesearch = scipy.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals],C_ICP.T))
    matches = treesearch.query(evecs_tar[:,0:n_evals], k=1)[1]
    W = np.linalg.lstsq(evecs_tar[matches, 0:n_evals],evecs_src[:, 0:n_evals])[0]
    d = np.linalg.svd(W)
    C_ICP = np.matmul(np.matmul(d[0], np.eye(n_evals)), d[2])

# C Visualization
plt.imshow(C_ICP)
plt.colorbar()
plt.show()

# Correspondence visualization 
treesearch = scipy.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals],C_ICP.T))
p2p_icp = treesearch.query(evecs_tar[:,0:n_evals], k=1)[1]

# Correspondence visualization
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p_icp,:]])
p.show()

# Computing (euclidean) error evaluation
err = np.sum(np.square(v_tar - v_tar[p2p_icp,:]))
print(err)

## ZoomOut refinement

Finally, a state of the art post-processing is ZoomOut. This lets us include much more frequencies without explicitly solving a costly optimization.

In [ ]:
# More iterations => bigger map => higher frequencies => better quality
n_iter = 10

# ZOOMOUT
C_iter = C_ICP
for i in np.arange(0,n_iter):
  # 1) Convert into dense correspondence
  treesearch = scipy.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals+i],C_iter.T))
  p2p_ = treesearch.query(evecs_tar[:,0:n_evals+i], k=1)[1]
 
  #2) Convert into C of dimension (n+1) x (n+1)
  C_iter = np.matmul(np.linalg.pinv(evecs_tar[:,0:n_evals+i+1]),evecs_src[p2p_,0:n_evals+i+1])


# Evaluate the map and visualize the result
plt.imshow(C_iter)
plt.colorbar()
plt.show()

# Correspondence visualization 
treesearch = sp.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals+n_iter],C_iter.T))
p2p_zo = treesearch.query(evecs_tar[:,0:n_evals+n_iter], k=1)[1]

# Correspondence visualization
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p_zo,:]])
p.show()

# Computing error evaluation
err = np.sum(np.square(v_tar - v_tar[p2p_zo,:]))
print(err)

To better appreciate the improvements, we can plot all the methods togheter and show a function with higher frequencies.

In [ ]:
# Correspondence visualization
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

# Higher frequencies function
funz_src  = np.cos(funz_src * 10)

p = plot_RGBmap([v_src, v_tar, v_tar, v_tar, v_tar], [f_src,f_tar, f_tar,f_tar,f_tar,f_tar],
                [funz_src, funz_src, funz_src[p2p,:],funz_src[p2p_icp,:],funz_src[p2p_zo,:]])
p.show()

From the left: 

Source - Target (GT) - Target (FMAP) - Target (FMAP + ICP) - Target (FMAP + ICP + ZO)